# Практическое задание 4 

# Мультиязычный тематический поиск

## курс "Математические методы анализа текстов"


### ФИО: Щербаков Василий Сергеевич

## Введение

### Постановка задачи:

В этом задании вам предстоит сделать свою небольшую мультиязычную поисковую систему. Проще всего строить мультиязычные системы, имея "параллельные" данные: словари или корпуса параллельных текстов. 

В задании необходимо, имея англо-русскую и агло-испанскую коллекции, обучить модель поиска модель поиска испанских текстов по русским.

Решение этого задания будет основано на тематическом моделировании, а именно подходе аддитивной регуляризации.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [bigartm](http://bigartm.org/)
 - [pymorphy2](https://pymorphy2.readthedocs.io/en/latest/)
 - [nltk](http://www.nltk.org/)


### Данные

Данные — записи выступлений конференции TED Talks на трёх языках. 

Все данные содержатся в архиве `ted_collection`. В папке содержится три подпапки `/en`, `/ru` и `/es`, каждая из которых соответствует коллекции для отдельного языка. Папка `parallel_info`  содержит информацию о связях документов между коллекциями. Файл `titles_file.json` содержит информацию о заголовках документов английской коллекции.

Ссылка для скачивания данных: [ссылка на гугл диск](https://drive.google.com/file/d/1B3kDfISvWnVpEet_CDa6oLNp028mEak-/view)

#### Импорт важных библиотек

In [100]:
# считывание
import os
import glob
import json

# предобработка
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import re

# тематическое моделирование
import artm

# change log style for artm
lc = artm.messages.ConfigureLoggingArgs()
lc.minloglevel = 3
lib = artm.wrapper.LibArtm(logging_config=lc)

# визуализация
import matplotlib.pyplot as plt
%matplotlib inline

# прочее 
import numpy as np

#### Несколько важных промежуточных функций

In [101]:
# загрузка коллекции
from lab4_utils import load_collection

# загрузка информации о параллельных документах
from lab4_utils import load_parallel_documents_info

# запись vowpal wabit файла специального формата
from lab4_utils import write_vw_lab4

# подсчёт позиции в выдаче переводов текстов
from lab4_utils import get_indexes_of_relevant_documents

## Предобработка данных (3 балла)

Перед тем как начать моделировать, необходимо предобработать данные. 

Заметим, что для английского языка некоторые этапы обработки не оказывают сильного влияния на модель (например, лемматизация). В русском языке одному слову может соответствовать огромное количество различных форм, поэтому без лемматизации невозможно получить модель с хорошим качеством.

#### Предобработка русских текстов


Считываем коллекцию:

In [102]:
RU_DATA_PATH = 'ted_collection/ru'
ru_collection = load_collection(RU_DATA_PATH, verbose=True)

Total number of documents: 2731

Some document examples: 
	Я не знаю почему, но меня постоянно поражает мысль, что два с половиной миллиарда людей в мире связа...
	Я хочу вам рассказать о том, что мы можем узнать из изучения генома живых людей и вымерших людей. Но...
	Меня всегда привлекали компьютеры и технологии, и я написал несколько приложений для iPhone, iPod To...
	Поднимите руку, если знаете хотя бы одного из этих людей. Ничего себе, почти все. Действительно, каж...
	Когда дело касается коррупции, на ум приходит определённый тип людей.
В бывшем СССР были люди, страд...

Some file names examples: 
	ru_0
	ru_1
	ru_10
	ru_100
	ru_1001


Выведите полностью несколько документов. Подумайте, какую информацию из них можно удалить на этапе предобработки, не ухудшив качество решения задачи.

In [103]:
ru_collection["ru_0"]

'Я не знаю почему, но меня постоянно поражает мысль, что два с половиной миллиарда людей в мире связаны друг с другом через Интернет, и что в любой момент времени более 30% населения мира может зайти в Интернет, чтобы учиться, творить и обмениваться информацией. И количество времени, которое каждый из нас тратит на всё это, также продолжает расти. Недавние исследования показали, что одно только молодое поколение тратит более 8-ми часов в день в Интернете. Мне как родителю 9-летней девочки это число кажется очень заниженным. (Смех)\nНо так же, как Интернет открыл мир для всех и каждого из нас, он открыл и каждого из нас миру. И все чаще, цена, которую нас просят заплатить за всю эту связь — наша личная жизнь. Сегодня многим из нас очень хотелось бы верить, что Интернет — частная территория, но это не так. И каждым кликом мыши и каждым прикосновением к экрану, мы, как Гензель и Гретель, оставляем крошки нашей личной информации везде, где путешествуем в цифровом лесу. Мы оставляем наши дн

In [104]:
ru_collection["ru_1001"]

'Когда дело касается коррупции, на ум приходит определённый тип людей.\nВ бывшем СССР были люди, страдающие манией величия. Одним из них был Сапармурат Ниязов. До самой смерти в 2006 году он был всемогущим руководителем Туркменистана, среднеазиатской страны, богатой газом. Он очень любил проводить реформы. Одна из них касалась календаря — некоторые месяцы он переименовал в честь себя и своей матери. Он тратил миллионы долларов на создание собственного культа личности. Предметом особой гордости и величия стала 14-метровая позолоченная скульптура самого себя, которая горделиво возвышается на центральной площади в столице и вращается вслед за движением Солнца. Он был немного странным человеком.\nТипичное клише — африканский диктатор, министр или чиновник. Вспомним Теодорина Обианга. Его отец всю жизнь был президентом Экваториальной Гвинеи — западно-африканского государства, которое с 1990-х экспортирует нефть на миллиарды долларов и до сих пор ужасно ущемляет права человека. Подавляющее б

Что явно просится на удаление - информация об авторских правах: "TED.com translations are made possible by volunteer translators. Learn more about the Open Translation Project.\n© TED Conferences, LLC. All rights reserved.'". Она везде одинакова и не ифнормативна для данной постановки задачи. Спецсимволы типа начала новой строки - \n также не нужны. Также не влияет тестовка каких-то звуков(аплодисменты) и благодарности слушателям. Причем все это не заивист от языка, для всех языков лишняя информация одиакова(по смысловой нагрузке, написание, естественно разное)

Помимо удаления специфичной информации, вам необходимо провести для русского языка следующие шаги предобработки :

1. Приведение документов к нижнему регистру.

2. Удаление всех символов кроме букв.
    1. Для некоторых способов выделения коллокаций (см. бонусную часть), может быть полезна информация о знаках препинания. Также она полезна при необходимости строить синтаксический разбор предложения.
    2. Вам может помочь функция sub из библиотеки re.
3. Токенизация документов.
    1. Воспользуйтесь стандартным методом .split, функцией split из библиотеки re или одним из токенайзеров библиотеки nltk.
4. Лемматизация документов.
    1. Воспользуйтесь библиотекой pymorphy2
    2. Шаги 3 и 4 можно выполнить вместе, воспользовавшись библиотекой mytem (или её обёрткой на python pymystem)
5. Удаление стоп-слов
    1. Базовый список стоп слов можно получить из модуля nltk.corpus

После выполнения всех шагов сохраните результат в словарь аналогичный ru_collection (ключи - названия файлов, значения - предобработанный документ в формате str).



In [109]:
COPYRIGHT_PHRASE = "TED.com translations are made possible by volunteer translators. Learn more about the Open Translation Project.\n© TED Conferences, LLC. All rights reserved"

morph_analyzers = {
        'russian': pymorphy2.MorphAnalyzer(),
        'english': WordNetLemmatizer(),
        'spanish': SnowballStemmer('spanish')
}

def process_documents(documents, language):
    #dicts for generic processing
    morph_analyzers_func ={
        'russian': lambda word: morph_analyzers['russian'].parse(word)[0].normal_form,
        'english': lambda word: morph_analyzers['english'].lemmatize(word),
        'spanish': lambda word: morph_analyzers['spanish'].stem(word)
    } 
    regex_all_except_letters = {
        'russian': r'[^а-яА-Я ,.]+',
        'english': r'[^a-zA-Z ,.]+',
        'spanish': r'[^a-zA-Z ,.]+'
    }
    words_to_cut ={
        'russian': ['аплодисменты', 'спасибо'],
        'english': ['applause', 'thank'],
        'spanish': ['aplausos', 'gracias'] 
    }
    processed_documents = {}
    for document_key, document_value in documents.items():
        # 0. - deleting unneded info
        processed_document = document_value[:document_value.index(COPYRIGHT_PHRASE)]
        # 1. - to low case
        processed_document = processed_document.lower()
        # 2. - deleting all chars except letters
        processed_document = re.sub(regex_all_except_letters[language],'', processed_document)
        # 3. - tokenize
        processed_document = re.split(r'[ ,.]+', processed_document)
        # 4. - lemmatize 
        processed_document = [morph_analyzers_func[language](word) for word in processed_document]
        # 5. - deleting stop words
        processed_document = filter(lambda word: word not in (stopwords.words(language) + words_to_cut[language]), processed_document)
        # adding result document to collection
        processed_documents [document_key] = ' '.join(processed_document)
    return processed_documents

Проанализируйте несколько предобработанных документов. Отметьте, какие неточности работы алгоритмов вы заметили, и как они могут повлиять на итоговую модель.

In [110]:
nltk.download("stopwords")
ru_collection_processed = process_documents(ru_collection, language = 'russian')

[nltk_data] Downloading package stopwords to /home/sh-
[nltk_data]     vasily/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [111]:
ru_collection_processed["ru_1001"]

'дело касаться коррупция ум приходить определнный тип человек бывший ссср человек страдать мания величие сапармурат нияз смерть год всемогущий руководитель туркменистан среднеазиатский страна богатый газ очень любить проводить реформа касаться календарь некоторый месяц переименовать честь свой мать тратить миллион доллар создание собственный культ личность предмет особый гордость величие стать метровый позолотить скульптура который горделиво возвышаться центральный площадь столица вращаться вслед движение солнце немного странный человек типичный клише африканский диктатор министр чиновник вспомнить теодорин обианга отец весь жизнь президент экваториальный гвинея западноафриканский государство который х экспортировать нефть миллиард доллар сей пора ужасно ущемлять право человек подавлять большинство человек живт ужасающий нищета хотя доход душа население португалия время обианга младший покупать особняк малиб калифорния миллион доллар побывать ворота поверьте размах внушительный купить 

#### Предобработка английских текстов

Считываем коллекцию:

In [112]:
EN_DATA_PATH = 'ted_collection/en'
en_collection = load_collection(EN_DATA_PATH, verbose=True)

Total number of documents: 2856

Some document examples: 
	I don't know why, but I'm continually amazed to think that two and a half billion of us around the w...
	What I want to talk to you about is what we can learn from studying the genomes of living people and...
	I've always had a fascination for computers and technology, and I made a few apps for the iPhone, iP...
	By raising your hand, how many of you know at least one person on the screen? Wow, it's almost a ful...
	I will never forget the first time I visited a client in jail. The heavy, metal door slammed behind ...

Some file names examples: 
	en_0
	en_1
	en_10
	en_100
	en_1000


Проведите все этапы предобработки для английского языка (шаг 4 опционален, можно использовать WordNetLemmatizer из nltk):

In [119]:
nltk.download('wordnet')
en_collection_processed = process_documents(en_collection, language = 'english')

[nltk_data] Downloading package wordnet to /home/sh-
[nltk_data]     vasily/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [120]:
en_collection_processed["en_100"]

'raising hand many know least one person screen wow almost full house true famous field know common died pancreatic cancer however although sad news also thanks personal story raised awareness lethal disease become third cause cancer death eight percent patient survive beyond five year thats tiny number especially compare breast cancer survival rate almost percent doesnt really come surprise diagnosed pancreatic cancer mean facing almost certain death sentence whats shocking though last year number hasnt changed bit much progress ha made type tumor make pancreatic cancer treatment effective biomedical entrepreneur like work problem seem impossible understanding limitation trying find new innovative solution change outcome first piece bad news pancreatic cancer pancreas middle belly literally depicted orange screen barely see remove organ front also surrounded many vital organ like liver stomach bile duct ability tumor grow organ reason pancreatic cancer one painful tumor type hardtorea

#### Предобработка испанских текстов 

Считываем коллекцию:

In [121]:
ES_DATA_PATH = 'ted_collection/es'
es_collection = load_collection(ES_DATA_PATH, verbose=True)

Total number of documents: 2809

Some document examples: 
	No sé por qué, pero siempre me asombro al pensar que 2 000 millones y medio de nosotros en todo el m...
	Hoy quiero contarles qué podemos aprender del estudio del genoma de personas vivas y de humanos exti...
	Siempre he tenido fascinación por los computadores y la tecnología, y he hecho algunas aplicaciones ...
	Levanten la mano si reconocen al menos una persona en la pantalla. Vaya, casi todo el mundo. Es cier...
	Nunca olvidaré la primera vez que visité a un cliente en la cárcel. La pesada puerta de metal se cer...

Some file names examples: 
	es_0
	es_1
	es_10
	es_100
	es_1000


Проведите все этапы предобработки для испанского языка (шаг 4 опционален, можно использовать SpanishStemmer из nltk). 

**Замечание.** Регулярное выражение \w из библиотеки re позволяет выделять буквы (в том числе буквы испанского алфавита).

In [122]:
es_collection_processed = process_documents(es_collection, language = 'spanish')

In [123]:
es_collection_processed["es_1000"]

'nunc olvid primer vez visit client crcel pes puert metal cerr detrs m escuch llav gir cerradur pis cement debaj m ten pelcul adhes hac desgarr com cint despeg caj cad vez mov pie nic conexin exterior ventanit demasi alta par mir hab mesit cuadr atornill pis dos sill metal cad lad primer vez entend visceral sol moment fugaz cmo prisin hac much aos promet m mism com abog defensor pblic nunc nunc olvid sentimient nunc hech inspir luch libert tod cad client com si fuer ma libert concept tan fundamental par psiqu estadounidens consagr nuestr constitucin embarg ee uu adict encarcel esclavitud hast encarcel masiv siempr sid as tod conoc nmer impact ee uu encarcel ms person per cpit casi cualqui nacin planet per quizs sep cualqui noch ee uu casi medi milln person sid conden nad duerm celd crcel madr padr hij hij estn ah razn sol razn pued pag preci libert preci llam fianz fianz cre realid com form liberacin condicional teor simpl establec fianz cantid algui podr pag algun pag dar incent par v

#### Дополнительная предобработка

Библиотека BigARTM имеет собственный формат документов для обработки, называемый батчами. Самый простой способ создать батчи из коллекции файлов - сконвертировать в батчи vowpal wabbit файл с коллекцией (https://github.com/VowpalWabbit/vowpal_wabbit/wiki/Input-format). 

Тематические модели работают с мешком слов, поэтому в vowpal wabbit файле можно не хранить информацию о порядке слов в документе.

Чтобы обучить мультиязычную модель, мы будем использовать апарат модальностей тематической модели. Каждому языку будет соответствовать своя модальность (`@english`, `@russian` и `@spanish` в vowpal wabbit файле). 

Для экспериментов нам понадобится два файла. Один будет содержать информацию о параллельных документов, а другой нет. В втором файле каждая строчка соответствует конкретному документу. В первом файле, если у документа есть параллельный, он будет располагаться на этой же строчке в рамках другой модальности.

Весь код для сохранения файла в vowpal wabbit формате уже написан, вам только необходимо правильно воспользоваться функцией. В частности, проследите, чтобы в ваших документах не содержалось символов ':', '|' и '@'.

В файлах en_ru_match.txt и en_es_match.txt содержится информация о том, какие документы являются параллельными. 

In [126]:
en_ru_parallel_docs = load_parallel_documents_info('ted_collection/parallel_info/en_ru_match.txt')
en_es_parallel_docs = load_parallel_documents_info('ted_collection/parallel_info/en_es_match.txt')

Total number of pairs: 1000
Total number of pairs: 1000


Воспользуйтесь функцией write_vw_lab4, чтобы сохранить данные в нужном формате:

In [127]:
# vowpal wabbit файлы с коллекцией
DATA_PATH_PARALLEL = 'ted_collection/collection_parallel.vw'
DATA_PATH_MONO = 'ted_collection/collection_mono.vw'

write_vw_lab4(
    DATA_PATH_PARALLEL,
    en_collection_processed,
    ru_collection_processed,
    es_collection_processed,
    en_ru_parallel_docs,
    en_es_parallel_docs,
    use_parallel_info=True
)


write_vw_lab4(
    DATA_PATH_MONO,
    en_collection_processed,
    ru_collection_processed,
    es_collection_processed,
    en_ru_parallel_docs,
    en_es_parallel_docs,
    use_parallel_info=False
)

## Базовая моноязычная тематическая модель (2 балла)

Теорию тематического моделирования можно узнать из соответствующей лекции курса.

Научиться пользоваться bigartm легче всего по гайду из документации [ссылка](http://docs.bigartm.org/en/stable/tutorials/python_userguide/index.html).

#### Подготовка к моделированию

Чтобы преобразовать полученный vowpal wabbit файл в батчи, можно воспользоваться стандартным классом BatchVectorizer. Объект этого класса принимает на вход адрес папки с батчами или файл vowpal wabbit, а затем подаётся на вход для обучения методам. В случае, если входные данные не являются батчами, он создаёт их и сохраняет на диск для последующего быстрого использования.

В этой части экспериментов, вам предлагается построить моноязычную тематическую модель только для английского языка, поэтому для обучения используйте файл DATA_PATH_MONO.

In [ ]:
# папка с батчами
BATCHES_PATH_MONO = ## your code here ##

In [ ]:
# если BATCHES_PATH_MONO пуста, батчи будут созданы из файла в DATA_PATH_MONO
# иначе использовать BATCHES_PATH_MONO
if len(glob.glob(os.path.join(BATCHES_PATH_MONO + '/*.batch'))) < 1:
    batch_vectorizer_mono = artm.BatchVectorizer(data_path=DATA_PATH_MONO, 
                                                 data_format='vowpal_wabbit',
                                                 target_folder=BATCHES_PATH_MONO)
else:
    batch_vectorizer_mono = artm.BatchVectorizer(data_path=BATCHES_PATH_MONO,
                                                 data_format='batches')

Словарь – это объект BigARTM, содержащий информацию о коллекции (словарь коллекции, различные величины и счётчики, связанные со словами). Создать словарь можно на основе папки с батчами. Затем собранный словарь можно сохранять на диск и позже подгрузить вновь.

In [ ]:
dictionary = artm.Dictionary()
dictionary.gather(data_path=BATCHES_PATH_MONO)

Словарь в том числе отвечает за то, на какие токены будет настраиваться модель. Редкие слова не оказывают влияние на модель, поэтому их можно удалить используя метод .filter.

In [ ]:
MIN_DF = 5

dictionary.filter(min_df=MIN_DF)

#### Обучение модели

Пришло время приступить к моделированию! Начнём с простой одноязычной модели PLSA. Будем учитывать только модальность @english в документах коллекции. Так как коллекция небольшая, используйте небольшое число тем 30-50.

Параметр theta_columns_naming='title' отвечает за именование документов лейблами из vowpal wabbit формата при получении матрицы $\Theta$ (иначе они будут нумероваться в порядке появления в коллекции).

In [ ]:
model = artm.ARTM(num_topics=30,
                  num_processors=7,
                  theta_columns_naming='title',
                  show_progress_bars=True,
                  class_ids={'@english':1})

model.initialize(dictionary)

Класс artm.ARTM поддерживает различные встроенные метрики качества. Добавьте метрики измерения перплексии, разреженности $\Phi$, разреженности $\Theta$ и счётчик топ слов. Не забудьте, что метрики должны соответствовать только модальности @english!

In [ ]:
## your code here ##

Проведите обучение модели с помощью метода fit_offline. Подберите необходимое число операций для сходимости в зависимости от значения перплексии:

In [ ]:
## your code here ##

Выведите график изменения перплексии в зависимости от итерации алгоритма, чтобы мы знали, что ваш алгоритм точно сошёлся :)

In [ ]:
## your code here ##

#### Интерпретация результатов

Основная особенность тематических моделей — интерпретируемость получаемых матриц $\Phi$ и $\Theta$. С помощью $\Phi$ для каждой темы можно узнать топ-слова, а с помощью $\Theta$ для каждой темы можно узнать топ-документы. 
Для того, чтобы получить матрицу $\Theta$, используйте метод .transform.

Для каждой темы выведите топ её слов ($\geq 20$) и топ заголовкой её документов ($\geq 5$). Попробуйте интерпертировать полученные темы, действительно ли темы получаются осмысленными?

In [ ]:
file_name_to_title = {}
with open('ted_collection/titles_file.json') as f:
    for line in f:
        file_name_to_title.update(json.loads(line))

In [ ]:
## your code here ##

In [ ]:
# А здесь ваш текст про интерпретацию модели :)

## Мультиязычная тематическая модель (5 баллов)

В этом пункте задания вам будет необходимо обучить мультиязычную тематическую модель.

В данном пункте вы будете реализовывать модель ML-TD (MultiLingual Parallel).

* каждый язык — отдельная модальность
* $\theta_{td}$ — общая для всех параллельных документов

Таким образом на обучении все параллельные документы записываются в одну строку в vowpal wabbit файле.

Оценивать качество модели мы будем на задаче поиска перевода текста. Вам будет необходимо оценить качество трёх переводов: с русского на английский, с испанского на английский и с русского на испанский.

Поиск документов будет устроен следующим образом. Будем для документа d на языке A считать близости со всеми документами на языке B и ранжировать документы языка B по этой близости. Для каждого документа посчитаем позицию истинного перевода документа в выдаче. Итоговая метрика — медиана или среднее таких позиций по всем документам.

Выделим множества документов, для которых не было известно информации об их переводе:

In [ ]:
ru_es_parallel_docs_test = load_parallel_documents_info('ted_collection/ru_es_match_test.txt', verbose=False)
ru_en_parallel_docs_test = load_parallel_documents_info('ted_collection/ru_en_match_test.txt', verbose=False)
es_en_parallel_docs_test = load_parallel_documents_info('ted_collection/es_en_match_test.txt', verbose=False)

Проведите подготовительный этап (создание батчей и словарей) для мультиязычной коллекции DATA_PATH_PARALLEL.

In [ ]:
## your code here ##

Обучите мультиязычную модель и проинтерпертируйте полученные темы. Если вы всё сделали правильно, то топ-слова различных языков для одной темы должны получиться достаточно похожими.

In [ ]:
## your code here ##

Протестируйте качество на трёх задачах перевода. Добейтесь хорошего качества (медиана позиции в выдаче ~ 0, среднее ~ 10). Получить положение в выдаче переводов текстов вам поможет функция get_indexes_of_relevant_documents из модуля lab4_utils. Для того, чтобы всё работало корректно (на тесте не должна быть известна информация о параллельности документов), подавайте в качестве theta результат model.transform(batch_vectorizer_mono).

Возможные способы улучшения:
* изменять веса модальностей
* изменять количество тем, количество итераций обучения
* добавлять регуляризаторы (см. бонусную часть)
* изменять метрику для поиска ближайших документов
* добавлять шаги в предобработку (выделение колокаций)

**В первую очередь** рекомендуется подобрать количество тем (в данной задаче хорошо работает небольшое число тем - несколько десятков) и веса модальностей.

За нетривиальные подходы могут быть начислены дополнительные бонусные баллы!

In [ ]:
## your code here ##

Проведите анализ ошибок. На каких документах модель отработала лучше всего, на каких хуже всего? Как вы думаете почему?

In [ ]:
# Здесь ваш текст про ошибки модели :)

## Бонусная часть

#### Добавление фоновых тем (до 1 балла)

Основной инструмент улучшения качества тематической модели — регуляризация. Выделите часть тем модели как фоновые. Сглаживайте $\Theta$ для фоновых тем, разреживайте для предметных.

Проинтерпретируйте результаты. Фоновые темы должны иметь в качество топ-слов слова фоновой лексики! Удалось ли с помощью введения фоновых тем повысить качество модели?

#### Модальность n-грамм (до 2 баллов)

Для каждого языка добавьте дополнительную модальность n-грамм. n-граммы можно выделить, например, с помощью пакета Phrases из модуля Gensim. Как отразилось добавление новой модальности на интерпретируемости модели? Удалось ли с помощью введения n-грамм повысить качество модели?